# Uvozim knjižice

In [5]:
import spacy
import nltk
from tqdm.notebook import tqdm
import networkx as nx
import re #regular expression

# Priprava besedila

In [6]:
#1 Naložimo besedilo
book=open("GOT.txt",encoding="utf8").read()

#2 Čiščenje besedila
book=re.sub(r"“","",book)
book=re.sub(r"”","",book)

#3 Diskretizacija besedila
stavki=nltk.sent_tokenize(book)
print(f"Stevilo stavkov v besedilu je {len(stavki)}")

Stevilo stavkov v besedilu je 89040


# NER (Named Enteties Recognition): Spacy

In [3]:
nlp=spacy.load("en_core_web_md")

<img src="https://user-images.githubusercontent.com/13643239/55229632-dbff9480-521d-11e9-8499-efb2a9c948db.png" alt="Slika_1" width="900"/>

In [7]:
liki={}

N=int(0.1*len(stavki))

for s in tqdm(range(N),total=N):
    doc=nlp(stavki[s])
    for ent in doc.ents:
        if ent.label_=="PERSON":
            oseba=ent.text
            if oseba not in liki:
                liki[oseba]=[]
                #print(s,oseba, ent.start_char, ent.end_char, ent.label_)
            liki[oseba].append(s)

  0%|          | 0/8904 [00:00<?, ?it/s]

In [8]:
for oseba in list(liki.keys())[:10]:
    print(oseba,liki[oseba])

Gared [0, 11, 20, 25, 49, 54, 58, 111, 114, 126, 138, 195, 212, 217]
Waymar Royce [3, 23, 47, 160, 173, 236, 301, 345, 373, 5980, 7885]
grey-eyed [48]
Waymar [51, 112, 126, 129, 133, 177, 213, 249, 253, 285, 312, 322, 328, 335, 342]
Mallisters [73]
m’lord [108, 178, 202, 4349, 4815, 7894, 8884]
Maester Aemon [135, 369, 5710, 5794, 6577, 6581, 6583, 6649, 8396, 8571]
Will [144, 170]
Royce [169]
wolf [174, 704, 1608, 1721, 2297, 2301, 2532, 2536, 2538, 2540, 2543, 2783, 3834, 4006, 4176, 4263, 4269, 4534, 4649, 4683, 4727, 5279, 7102, 7843]


# Ustvarjanje sociograma

## Mrežo poselimo z vozlišli, ki ponazarjajo posamezne like

In [9]:
G=nx.Graph()
for oseba in liki:
    G.add_node(oseba)

print(f"Število oseb = {len(G)}")

nodes=G.nodes()

Število oseb = 294


In [10]:
for p1 in tqdm(G,total=len(G)):
    x=np.array(liki[p1])
    for p2 in G:
        y=np.array(liki[p2])
        if p1!=p2:
            xy=np.array([np.min(np.abs(y-xi)) for xi in x])
            wij=len(np.where(xy<5)[0])
            if wij>3:
                #print(p1,p2,wij)
                G.add_edge(p1,p2,weight=wij)

  0%|          | 0/294 [00:00<?, ?it/s]

In [11]:
remove_nodes=[]
for node in G:
    if G.degree(node)<1:
        remove_nodes.append(node)

for node in remove_nodes:
    G.remove_node(node)

nx.write_graphml(G,"sociogram.graphml")

# Kateri hiši pripadajo liki?

appendix=open("appendix.txt",encoding="utf8").read().split("Other Houses Great And Small")[1]
appendix=re.sub("\n\n+","\n",appendix)

house=None
pattern = r"\b[A-Z]+\b"
for p in appendix.split("\n")[1::]:
    if "HOUSE" in p:
        house=p.split("HOUSE")[1]
    elif p[0]=="—":
        for pp in p.split(","):
            matches = ' '.join(re.findall(pattern, pp))
            if matches!='':
                print(house,matches)

# Dodatna analiza besedila

In [13]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
import pandas as pd

In [20]:
print(nlp.pipe_names)

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner', 'spacytextblob']


In [ ]:
patterns = [
    {
        "pattern": [{"LOWER": {"IN": ["sword", "swords", "dagger", "daggers"]}}],
        "attrs": {"_": {"is_weapon": True}},
    },
]

attribute_ruler.add_patterns(patterns)

In [18]:
for s in tqdm(range(N),total=N):
    doc=nlp(stavki[s])
    s=doc._.blob.polarity
    if abs(s)>0.5:
        print(f"{doc} | {round(s,2)}")

  0%|          | 0/8904 [00:00<?, ?it/s]

He had laughed about it afterward. | 0.7
Today was the worst of all. | -1.0
A cold wind was blowing out of the north, and it made the trees rustle like living things. | -0.6
Heavy-looking, double-bladed, a cruel piece of iron. | -1.0
Ser Waymar asked casually. | -0.5
It was the cold, Gared said with iron certainty. | -0.6
Nothing burns like the cold. | -0.6
I’ve had the cold in me too, lordling. | -0.6
You ought dress more warmly, Gared. | 0.55
We’ll see how warm you can dress when the winter comes. | 0.6
There never was a week when he did not draw a dozen bloody watches. | -0.8
Bright lad. | 0.7
A half-moon rose. | 0.6
Best go the rest of the way on foot, m’lord. | 1.0
A cold wind whispered through the trees. | -0.6
Four years in the Night’s Watch, and he had never been so afraid. | -0.6
It was a splendid weapon, castle-forged, and new-made from the look of it. | 0.83
Will doubted it had ever been swung in anger. | -0.7
The taste of cold iron in his mouth gave him comfort. | -0.6
Why 

If she did marry Prince Joff, Arya hoped for his sake that he had a good steward. | 0.7
Nymeria was waiting for her in the guardroom at the base of the stairs. | -0.8
The wolf pup loved her, even if no one else did. | 0.7
That had been a great scandal too. | 0.8
He smiled. | 0.6
I could do just as good as Bran, she said. | 0.7
No doubt the needlework was exquisite. | 1.0
The Lannisters are proud, Jon observed. | 0.8
Are you afraid? | -0.6
Some of the Lannister men laughed. | 0.7
You’ll be sorry! | -0.62
I hate needlework! | -1.0
Nothing is fair, Jon said. | 0.7
That was almost as good as going south with the king. | 0.7
How could he be afraid? | -0.6
The greatest living knight was Ser Barristan Selmy, Barristan the Bold, the Lord Commander of the Kingsguard. | 0.67
Bran did his best, although he did not think he ever really fooled her. | 0.6
That was the best time of all. | 1.0
That was another thing he liked about climbing; it was almost like being invisible. | 0.6
Bran hung, listenin

She heard her saying, The council does us great honor, my good lords. | 0.75
When she got closer, she saw two knights kneeling before the queen, in armor so fine and gorgeous that it made her blink. | 0.56
Cradled under one arm was an antlered helm, its magnificent rack shimmering in gold. | 1.0
He stood to one side, beside their horses, a gaunt grim man who watched the proceedings in silence. | -1.0
Still, Sansa wrenched away from him, and the Hound laughed, and Lady moved between them, rumbling a warning. | 0.7
Why are you afraid? | -0.6
The honor is mine, good knight. | 0.7
He smiled at her. | 0.6
Then she smiled graciously and said, Sansa, the good councillors and I must speak together until the king returns with your father. | 0.65
Joffrey, perhaps you would be so kind as to entertain our guest today. | 0.6
The way he had rescued her from Ser Ilyn and the Hound, why, it was almost like the songs, like the time Serwyn of the Mirror Shield saved the Princess Daeryssa from the giants

KeyboardInterrupt: 